<span STYLE="font-size:200%"> 
    Data analysis of the BOSE uniaxial compression test
</span>

Docker image: gnasello/bose-compression:2023-03-29 \
Conda environment: Python3 (ipykernel) \
Latest update: 29 March 2023

# Import libraries

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as sfa
import numpy as np
from matplotlib.lines import Line2D
from pathlib import Path

# Load data

In [ ]:
filename = 'example/2mgml-noX-1 03282023 013109_tdf.CSV'

In [ ]:
# skiprows - the first three lines of tdf.CSV files are not relevant for the dataframe object 
df_original = pd.read_csv(filename, skiprows=3) 

units = df_original.iloc[0]
units

In [ ]:
df_original.head()

Remove not relevant data

In [ ]:
df = df_original.drop([0])
df = df.iloc[:, :-1].astype(float) # drop last column
df

# Select interval of interest

<span style="color:red">**User Input**</span>

In [ ]:
time_var = 'Elapsed Time'
test_duration = 2 #seconds

In [ ]:
df = df[df[time_var] <= test_duration]
df

# Convert Load (in grams) to Force (in milli Newtons) and change sign

In [ ]:
g_acc = 9.80665 # m/s2

df['Load_g'] = df['Load 3']

df['Force'] = df['Load_g'] * (-g_acc) # g * m/s^2 = mN (10^-3 N)
df

# Time plots

In [ ]:
from matplotlib import rcParams

# figure size in inches
rcParams['figure.figsize'] = 5,5

g = sns.lineplot(data=df, x="Elapsed Time", y="Load 3", color="g")
plt.xlabel('Time (s)')
plt.ylabel('Load (g)')

sns.lineplot(data=df, x="Elapsed Time", y="Disp", color="r", ax=g.axes.twinx())
plt.ylabel('Displacement (mm)')
g.legend(handles=[Line2D([], [], marker='_', color="g", label='Load'), 
                  Line2D([], [], marker='_', color="r", label='Displacement')])

# Start from zero force

Keep only points where the force is greater than zero. This corresponds to when the mechanical compression started

In [ ]:
df = df[df['Force']>0]
df

In [ ]:
df['displacement'] = df['Disp'].iloc[0] - df['Disp']
df

# Force - Displacement plot

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
sns.scatterplot(data=df, x="displacement", y="Force", ax=ax)
plt.ylabel('Force (mN)')
plt.xlabel('Compressive Displacement (mm)')

# Stress-strain plot

## Convert forces to stresses (Pa)

<span style="color:red">**User Input**</span>

In [ ]:
radius = 5 # mm

sample_area = np.pi * (radius**2) # mm^2

df['stress'] = df['Force']/sample_area * 10**3 # mN / mm^2 * 10**3 = Pa
df

## Convert displacements to strains (%)

<span style="color:red">**User Input**</span>

In [ ]:
sample_thickness = 2 # mm

df['strain'] = df['displacement']/sample_thickness #
df['strain100'] = df['strain'] * 100
df

## Compute Young modulus

Young's modulus or the initial modulus (IM) is a measure of the amount of deformation that is caused by a small stress. 
[reference](https://www.sciencedirect.com/topics/engineering/initial-modulus)

In this case, we compute the Young modulus in the range of 0-5% strain.

In [ ]:
strain_threshold = 5 # %

df_young = df[ df['strain100'] < strain_threshold ]

In [ ]:
r = sfa.ols('stress ~ strain + 0', data=df_young).fit()
r.summary()

In [ ]:
young_pa = r.params[0] # Pa
young_kpa = young_pa / 1000 #kPa

print('--------------\n\n\nYoung modulus {0:.2f} kPa\n\n\n--------------'.format(young_kpa))

## Stress-strain plot

In [ ]:
x = df_young['strain100']

fig, ax = plt.subplots(figsize=(5, 5))
sns.scatterplot(data=df.sample(n=100, random_state=1), x="strain100", y="stress", ax=ax)
ax.plot(x, r.fittedvalues, linestyle='dashed', linewidth=2, color='black')
plt.ylabel('Stress (Pa)')
plt.xlabel('Compressive Strain (%)')

# https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.savefig.html
plt.savefig('Stress_Strain_curve.svg')

# Save data

Save new `.csv` file in the same directory where the orginal dataset is located

In [ ]:
newfile = Path(filename).parent / (Path(filename).stem + '_analyzed.csv')
print(newfile)

In [ ]:
df.to_csv(newfile)